<a href="https://colab.research.google.com/github/reparadise/MachinelearningProjectAifel/blob/main/EXPLORATION5_%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EA%B3%BC_%EA%B0%80%EC%9C%84%EB%B0%94%EC%9C%84%EB%B3%B4_%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXPLORATION 5. 인공지능과 가위바위보 하기

---

##  Dataset

### Data 만들기

teachable machine 사이트에서 데이터를 만들어 냅니다.

[teachablemachine](https://teachablemachine.withgoogle.com/)

### Resize 하기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from PIL import Image
import os, glob
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/content/drive/MyDrive/Colab Notebooks/Data/data/sissors"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")


# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
# [[YOUR CODE]]
rock_dir_path = os.getenv("HOME") + "/content/drive/MyDrive/Colab Notebooks/Data/data/rocks"
rock_images = glob.glob(rock_dir_path + "/*.jpg")
# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.

target_size=(28,28)
for img in rock_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")


print("바위 이미지 resize 완료!")


# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
# [[YOUR CODE]]
paper_dir_path = os.getenv("HOME") + "/content/drive/MyDrive/Colab Notebooks/Data/data/papers"
paper_images = glob.glob(paper_dir_path + "/*.jpg")
# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
# [[YOUR CODE]]
target_size = (28,28)
for img in paper_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
    
print("보 이미지 resize 완료!")

이미지 디렉토리 경로:  /root/content/drive/MyDrive/Colab Notebooks/Data/data/sissors
가위 이미지 resize 완료!
이미지 디렉토리 경로:  /root/content/drive/MyDrive/Colab Notebooks/Data/data/sissors
바위 이미지 resize 완료!
이미지 디렉토리 경로:  /root/content/drive/MyDrive/Colab Notebooks/Data/data/sissors
보 이미지 resize 완료!


In [13]:

import matplotlib.pyplot as plt

### Data 불러오기

In [7]:
import numpy as np
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=6600   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels=np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 0 입니다.
x_train shape: (6600, 28, 28, 3)
y_train shape: (6600,)


## Model

### model 설계하기

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3200)              0         
                                                                 
 dense (Dense)               (None, 64)                2

### 모델 학습하기

In [9]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.compile()과 model.fit()을 사용해 봅시다.
# [[YOUR CODE]]
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train , epochs=10)

Epoch 1/10
207/207 [==============================] - 15s 70ms/step - loss: 0.9705 - accuracy: 1.0000
Epoch 2/10
207/207 [==============================] - 14s 69ms/step - loss: 0.7489 - accuracy: 1.0000
Epoch 3/10
207/207 [==============================] - 14s 69ms/step - loss: 0.5784 - accuracy: 1.0000
Epoch 4/10
207/207 [==============================] - 14s 69ms/step - loss: 0.4502 - accuracy: 1.0000
Epoch 5/10
207/207 [==============================] - 14s 69ms/step - loss: 0.3545 - accuracy: 1.0000
Epoch 6/10
207/207 [==============================] - 14s 69ms/step - loss: 0.2829 - accuracy: 1.0000
Epoch 7/10
207/207 [==============================] - 16s 78ms/step - loss: 0.2288 - accuracy: 1.0000
Epoch 8/10
207/207 [==============================] - 14s 69ms/step - loss: 0.1875 - accuracy: 1.0000
Epoch 9/10
207/207 [==============================] - 14s 69ms/step - loss: 0.1555 - accuracy: 1.0000
Epoch 10/10
207/207 [==============================] - 14s 69ms/step - loss: 0.130

### Test data 불러오기

In [15]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
# [[YOUR CODE]]
def load_test(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=900   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels=np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    return imgs, labels

In [16]:
test_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
print(test_image_dir_path)

# scissor
t_s_images=glob.glob(test_image_dir_path + "/scissor/*.jpg")  
target_size=(28,28)
for img in t_s_images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")

# rock
t_r_images=glob.glob(test_image_dir_path + "/rock/*.jpg")  
target_size=(28,28)
for img in t_r_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
 
print("바위 이미지 resize 완료!")

# paper
t_p_images=glob.glob(test_image_dir_path + "/paper/*.jpg")  
target_size=(28,28)
for img in t_p_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
 
print("바위 이미지 resize 완료!")


(x_test, y_test)=load_test(test_image_dir_path)
x_test_norm = x_test/255.0

print(x_test.shape)

/root/aiffel/rock_scissor_paper/test
가위 이미지 resize 완료!
바위 이미지 resize 완료!
바위 이미지 resize 완료!
(900, 28, 28, 3)


### Model Test 하기

In [17]:
# model을 학습시키는 코드
#model.evaluate()

test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

29/29 - 1s - loss: 0.1194 - accuracy: 1.0000 - 630ms/epoch - 22ms/step
test_loss: 0.11937019973993301 
test_accuracy: 1.0


## Report

### 초기 데이터셋

test acc 는 1 


### 